In [1]:
import wntr
import numpy as np
import copy
import networkx as nx
import pickle

In [2]:
import matplotlib
import matplotlib.pyplot as plt
# "the default sans-serif font is Arial"
matplotlib.rcParams['font.sans-serif'] = "Arial"
# Then, "ALWAYS use sans-serif fonts"
# matplotlib.rcParams['font.family'] = "sans-serif"
matplotlib.rcParams.update({'font.size': 14})

In [26]:
# nf is the normalized factor for the weight estimation
def build_directed_network(wdn, flow, t):
    # Build directed graphs 
    dgraph = nx.DiGraph()
    for name, link in wdn.links():
        # Determine the direction of edge through the flow rate, positive: as stored, negative: change direction
        linkflow = flow.loc[3600*t, name]
        if linkflow >= 0:
            start_node = link.start_node_name
            end_node = link.end_node_name
        else:
            start_node = link.end_node_name
            end_node = link.start_node_name
        try:
            link_length = link.length
            link_diameter = link.diameter 
        # If it is a pump
        except:
            link_length = 1
            link_diameter = 0.762
        # Build corresponding original network
        dgraph.add_node(start_node, pos = wdn.get_node(start_node).coordinates)
        dgraph.add_node(end_node, pos = wdn.get_node(end_node).coordinates)
        dgraph.add_edge(start_node, end_node, linkid = name, linkweight = link_length/link_diameter)
        # ugraph = dgraph.to_undirected()
    return dgraph 

In [27]:
# Get the non-zero expected demand
def get_demand_nodes(wdn):
    junctions = wdn.junction_name_list
    demand_nodes = []
    for j in junctions:
        j_object = wdn.get_node(j)
        base_demand = j_object.demand_timeseries_list[0].base_value
        if base_demand > 1e-8:
            demand_nodes.append(j)
    return demand_nodes

In [28]:
def surrogate(state_to_check, pipe_list, wdn, flow, demand, nodes, seriest, sp_threshold):
    # Initialization
    reservoirs = wdn.reservoir_name_list
    tanks = wdn.tank_name_list
    sources = reservoirs + tanks  
    sp_nodes = {}
    new_sp_nodes = {}
    dgraph_t = {}
    for t in seriest:
        sp_nodes[t] = {}
        new_sp_nodes[t] = {}
        for n in nodes:
            sp_nodes[t][n] = 1e6 # assign a large value
            new_sp_nodes[t][n] = 1e6
        for tn in tanks:
            sp_nodes[t][tn] = 1e6
            new_sp_nodes[t][tn] = 1e6
    
    # Get the original shortest path length
    for t in seriest: 
        dgraph = build_directed_network(wdn, flow, t)
        dgraph_t[t] = dgraph 
        # Get all the shortest paths from all the possible supply sources to the demand nodes
        for s in sources:
            sspt = nx.shortest_path_length(dgraph, source = s, weight = 'linkweight')
            # Update the shortest path to each demand node at this time instant
            for n in nodes:
                # If the node is reachable from the source
                if n in sspt:
                    # Get the shortest path
                    sp_nodes[t][n] = min(sp_nodes[t][n], sspt[n])
        # Get all the shortest paths from reservoirs to the tanks 
        for s in reservoirs:
            sspt = nx.shortest_path_length(dgraph, source = s, weight = 'linkweight')
            # Update the shortest path to each tank at this time instant
            for tn in tanks:
                # If the tank is reachable from the source
                if tn in sspt:
                    sp_nodes[t][tn] = min(sp_nodes[t][tn], sspt[tn])
    
    # Get the new state s-t shortest path length
    # The corresponding pipe off list
    pipe_off = []
    for i in range(len(state_to_check)):
        pipe_off.append(pipe_list[state_to_check[i]])       
            
    for t in seriest: 
        dgraph = copy.deepcopy(dgraph_t[t])
        # Get the edge to linkid mapping
        edge_id = nx.get_edge_attributes(dgraph, 'linkid')
        id_edge = {}
        # The edge to ppie mapping
        for key, value in edge_id.items():
            id_edge[value] = key
        # Remove the pipes
        # For the parallel pipes, only one pipe is recorded in the directed graph
        for p in pipe_off:
            try:
                edge = id_edge[p]
                dgraph.remove_edge(edge[0], edge[1])
            except:
                pass
        # Get all the shortest paths from all supply sources
        for s in sources:
            sspt = nx.shortest_path_length(dgraph, source = s, weight = 'linkweight')
            # Update the shortest path to each demand node at this time instant
            for n in nodes:
                # If the node is reachable from the source
                if n in sspt:
                    # Get the shortest path
                    new_sp_nodes[t][n] = min(new_sp_nodes[t][n], sspt[n])
        # Get all the shortest paths from reservoirs to the tanks 
        for s in reservoirs:
            sspt = nx.shortest_path_length(dgraph, source = s, weight = 'linkweight')
            # Update the shortest path to each tank at this time instant
            for tn in tanks:
                # If the tank is reachable from the source
                if tn in sspt:
                    new_sp_nodes[t][tn] = min(new_sp_nodes[t][tn], sspt[tn])
    
    # If the ratio of new shortest path length to the original is larger than the threshold,
    # we assume that the water is not served
    actual_demand = 0
    expect_demand = 0
    for t in seriest:
        for n in nodes:
            expect_demand += max(demand.loc[3600*t, n], 0)
            if (new_sp_nodes[t][n] / sp_nodes[t][n]) < sp_threshold:
                actual_demand += max(demand.loc[3600*t, n], 0)
        for tn in tanks:
            expect_demand += max(demand.loc[3600*t, tn], 0)
            if (new_sp_nodes[t][tn] / sp_nodes[t][tn]) < sp_threshold:
                actual_demand += max(demand.loc[3600*t, tn], 0)
    # The average functionality loss over the whole time period
    average_fl = 1 - (actual_demand / expect_demand)
    return average_fl

In [37]:
Wdn_name = 'WCRINS'
Wdn = wntr.network.WaterNetworkModel(Wdn_name + '.inp')
WdnC = copy.deepcopy(Wdn) 
WdnC.options.time.duration = 32 * 3600
WdnC.options.hydraulic.demand_model = 'PDA'
Sim = wntr.sim.WNTRSimulator(WdnC)
Results = Sim.run_sim() # by default run EPANET 2.2
Flows = Results.link['flowrate']
Demands = Results.node['demand'] 

In [38]:
Fpipelist = open('WCRINS_Pipelist.pickle','rb')
Pipelist = pickle.load(Fpipelist)
Fpipelist.close()
Seriest = np.linspace(9, 32, 24)
Nodes = get_demand_nodes(Wdn)
Sp_threshold = 2

In [39]:
# Two link failure
Pipe_afl = {}
for i in range(len(Pipelist)):
    for j in range((i+1), len(Pipelist)):
        Afl = surrogate([i,j], Pipelist, Wdn, Flows, Demands, Nodes, Seriest, Sp_threshold)
        Pipe_afl[(Pipelist[i], Pipelist[j])] = Afl 

In [40]:
FPipe_afl = open(Wdn_name + 'N2_Surrogate.pickle','wb')
pickle.dump(Pipe_afl, FPipe_afl)
FPipe_afl.close()

In [23]:
# Single link failure
Pipe_afl = {}
for i in range(len(Pipelist)):
    Afl = surrogate([i], Pipelist, Wdn, Flows, Demands, Nodes, Seriest, Sp_threshold)
    Pipe_afl[Pipelist[i]] = Afl 